In [19]:
"""
# Python 2.7
# Filename: apriori.py
# Author: llhthinker
# Email: hangliu56[AT]gmail[DOT]com
# Blog: http://www.cnblogs.com/llhthinker/p/6719779.html
# Date: 2017-04-16
"""


def load_data_set():
    """
    Load a sample data set (From Data Mining: Concepts and Techniques, 3th Edition)
    Returns: 
        A data set: A list of transactions. Each transaction contains several items.
    """
    data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'],
            ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]
    return data_set


def create_C1(data_set):
    """
    Create frequent candidate 1-itemset C1 by scaning data set.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
    Returns:
        C1: A set which contains all frequent candidate 1-itemsets
    """
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1


def is_apriori(Ck_item, Lksub1):
    """
    Judge whether a frequent candidate k-itemset satisfy Apriori property.
    Args:
        Ck_item: a frequent candidate k-itemset in Ck which contains all frequent
                 candidate k-itemsets.
        Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
    Returns:
        True: satisfying Apriori property.
        False: Not satisfying Apriori property.
    """
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True


def create_Ck(Lksub1, k):
    """
    Create Ck, a set which contains all all frequent candidate k-itemsets
    by Lk-1's own connection operation.
    Args:
        Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
        k: the item number of a frequent itemset.
    Return:
        Ck: a set which contains all all frequent candidate k-itemsets.
    """
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    for i in range(len_Lksub1):
        for j in range(1, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck


def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    """
    Generate Lk by executing a delete policy from Ck.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
        Ck: A set which contains all all frequent candidate k-itemsets.
        min_support: The minimum support.
        support_data: A dictionary. The key is frequent itemset and the value is support.
    Returns:
        Lk: A set which contains all all frequent k-itemsets.
    """
    Lk = set()
    item_count = {}
    for t in data_set:
        for item in Ck:
            if item.issubset(t):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    t_num = float(len(data_set))
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk


def generate_L(data_set, k, min_support):
    """
    Generate all frequent itemsets.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
        k: Maximum number of items for all frequent itemsets.
        min_support: The minimum support.
    Returns:
        L: The list of Lk.
        support_data: A dictionary. The key is frequent itemset and the value is support.
    """
    support_data = {}
    C1 = create_C1(data_set)
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    Lksub1 = L1.copy()
    L = []
    L.append(Lksub1)
    for i in range(2, k+1):
        Ci = create_Ck(Lksub1, i)
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        Lksub1 = Li.copy()
        L.append(Lksub1)
    return L, support_data


def generate_big_rules(L, support_data, min_conf):
    """
    Generate big rules from frequent itemsets.
    Args:
        L: The list of Lk.
        support_data: A dictionary. The key is frequent itemset and the value is support.
        min_conf: Minimal confidence.
    Returns:
        big_rule_list: A list which contains all big rules. Each big rule is represented
                       as a 3-tuple.
    """
    big_rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and big_rule not in big_rule_list:
                        # print freq_set-sub_set, " => ", sub_set, "conf: ", conf
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list


if __name__ == "__main__":
    """
    Test
    """
    data_set = load_data_set()
    L, support_data = generate_L(data_set, k=3, min_support=0.2)
    big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)
    for Lk in L:
        print ("="*50)
        print ("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
        print ("="*50)
        for freq_set in Lk:
            print (freq_set, support_data[freq_set])
    
    print ("Big Rules")
    for item in big_rules_list:
        print (item[0], "=>", item[1], "conf: ", item[2])
    print (L[1])


frequent 1-itemsets		support
frozenset({'l2'}) 0.7777777777777778
frozenset({'l4'}) 0.2222222222222222
frozenset({'l1'}) 0.6666666666666666
frozenset({'l5'}) 0.2222222222222222
frozenset({'l3'}) 0.6666666666666666
frequent 2-itemsets		support
frozenset({'l1', 'l2'}) 0.4444444444444444
frozenset({'l1', 'l3'}) 0.4444444444444444
frozenset({'l3', 'l2'}) 0.4444444444444444
frozenset({'l4', 'l2'}) 0.2222222222222222
frozenset({'l5', 'l1'}) 0.2222222222222222
frozenset({'l5', 'l2'}) 0.2222222222222222
frequent 3-itemsets		support
frozenset({'l5', 'l1', 'l2'}) 0.2222222222222222
frozenset({'l1', 'l3', 'l2'}) 0.2222222222222222
Big Rules
frozenset({'l4'}) => frozenset({'l2'}) conf:  1.0
frozenset({'l5'}) => frozenset({'l1'}) conf:  1.0
frozenset({'l5'}) => frozenset({'l2'}) conf:  1.0
frozenset({'l5', 'l1'}) => frozenset({'l2'}) conf:  1.0
frozenset({'l5', 'l2'}) => frozenset({'l1'}) conf:  1.0
frozenset({'l5'}) => frozenset({'l1', 'l2'}) conf:  1.0
{frozenset({'l1', 'l2'}), frozenset({'l1', '

In [2]:
C1 = create_C1(data_set)
print(C1)

{frozenset({'l3'}), frozenset({'l2'}), frozenset({'l5'}), frozenset({'l1'}), frozenset({'l4'})}


In [3]:
L1 = generate_Lk_by_Ck(data_set, C1, 0.2, support_data)
print(L1)

{frozenset({'l3'}), frozenset({'l2'}), frozenset({'l5'}), frozenset({'l1'}), frozenset({'l4'})}


In [4]:
Lksub1 = L1.copy()
L = []
L.append(Lksub1)
print(L)

[{frozenset({'l3'}), frozenset({'l5'}), frozenset({'l1'}), frozenset({'l4'}), frozenset({'l2'})}]


In [5]:
C2 = create_Ck(Lksub1, 2)
print(C2)

{frozenset({'l3', 'l2'}), frozenset({'l2', 'l4'}), frozenset({'l3', 'l1'}), frozenset({'l3', 'l4'}), frozenset({'l3', 'l5'}), frozenset({'l1', 'l4'}), frozenset({'l5', 'l1'}), frozenset({'l5', 'l4'}), frozenset({'l2', 'l5'}), frozenset({'l2', 'l1'})}


In [7]:
L2 = generate_Lk_by_Ck(data_set, C2, 0.2, support_data)
print(L2)

{frozenset({'l3', 'l2'}), frozenset({'l2', 'l4'}), frozenset({'l3', 'l1'}), frozenset({'l5', 'l1'}), frozenset({'l2', 'l5'}), frozenset({'l2', 'l1'})}


In [8]:
print(support_data)

{frozenset({'l2'}): 0.7777777777777778, frozenset({'l5'}): 0.2222222222222222, frozenset({'l1'}): 0.6666666666666666, frozenset({'l4'}): 0.2222222222222222, frozenset({'l3'}): 0.6666666666666666, frozenset({'l5', 'l1'}): 0.2222222222222222, frozenset({'l2', 'l5'}): 0.2222222222222222, frozenset({'l2', 'l1'}): 0.4444444444444444, frozenset({'l2', 'l4'}): 0.2222222222222222, frozenset({'l3', 'l2'}): 0.4444444444444444, frozenset({'l3', 'l1'}): 0.4444444444444444, frozenset({'l2', 'l5', 'l1'}): 0.2222222222222222, frozenset({'l3', 'l2', 'l1'}): 0.2222222222222222}


In [9]:
print(data_set)

[['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'], ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'], ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]


In [12]:
frozenset({'l5', 'l1'}).issubset(['l1', 'l2', 'l5'])


True

In [14]:
item_count = {}
item = frozenset({'l5', 'l1'})
if item not in item_count:
                    item_count[item] = 1
else:
                    item_count[item] += 1
print(item_count)

{frozenset({'l5', 'l1'}): 1}


In [37]:
class Node:
    def __init__(self,val=[]):
        self.val = val
        self.children = {}
 
class HashTree:
    def __init__(self):
        self.root = Node()#建立根节点
        
    def insert(self,val):#给定一个值val,创建一个值为val的节点并将其插入到树中
        root = self.root
        i = 0
        while root.children:#if root has children
            if (val[i]-1) % 3 in root.children.keys():#如果想找的孩子存在
                root = root.children[(val[i]-1)%3]
                i += 1
            else:#if the child doesn't exist
                root.children[(val[i]-1)%3] = Node([val])
                return
                
        
        #so the root doesn't have child
        if len(root.val) < 3:#the value of root is smaller than 3
            root.val.append(val)
            return
        else:#root的值的个数已经等于3了，那么我们必须split root
            root.val.append(val)#先把val加入到root的值当中
            for item in root.val:#对于root值中的的每一项
                j = (item[i]-1) % 3#表示余数是多少
                if j in root.children.keys():#如果余数已经在其中了
                    root.children[j].val.append(item)#将item加入到列表中
                else:
                    root.children[j] = Node()#首先创建该节点
                    root.children[j].val = [item]#如果没有，则创建新列表
                root.val = []                
 
                if len(root.children[j].val) == 4:#分裂节点后，仍然出现了大于3的情况，那么我们需要继续分裂节点
                    i += 1
                    root = root.children[j]
                    for item in root.val:#对于root值中的的每一项
                        j = (item[i]-1) % 3#表示余数是多少
                        if j in root.children.keys():#如果余数已经在其中了                            
                            root.children[j].val.append(item)#将item加入到列表中
                        else: 
                            root.children[j] = Node()
                            root.children[j].val = [item]#如果没有，则创建新列表
                    root.val = []
                        
    def PrintTree(self,node):#如何 层次的输出树
        if not node.val:#如果这个节点的值不存在
            if not node.children: return#如果孩子也没有
            else:#有孩子了
                res = []
                for item in node.children.values():
                    res += [self.PrintTree(item)]
        else:#这个节点的值存在
            return node.val
        return res
    
                
                
            
        
 
        
obj = HashTree()
values = '[{1 2 4}, {1 2 9}, {1 3 5}, {1 3 9}, {1 4 7}, {1 5 8}, {1 6 7}, {1 7 9}, {1 8 9}, {2 3 5}, {2 4 7}, {2 5 6}, {2 5 7}, {2 5 8}, {2 6 7}, {2 6 8}, {2 6 9}, {2 7 8}, {3 4 5}, {3 4 7}, {3 5 7}, {3 5 8}, {3 6 8}, {3 7 9}, {3 8 9}, {4 5 7}, {4 5 8}, {4 6 7}, {4 6 9}, {4 7 8}, {5 6 7}, {5 7 9}, {5 8 9}, {6 7 8}, {6 7 9}]' 
values = values.replace('{','[').replace('}',']').replace(' ',',').replace(',,',',')
values = 'values = ' + values
 
exec(values)
 
for item in values:
    obj.insert(item)
 
 
print(obj.PrintTree(obj.root))



[[[[[1, 2, 4], [4, 5, 7]], [[1, 2, 9], [1, 8, 9]], [[1, 5, 8], [4, 5, 8]]], [[[1, 3, 5]], [[1, 3, 9], [4, 6, 9]], [[1, 6, 7], [4, 6, 7]]], [[1, 4, 7], [1, 7, 9], [4, 7, 8]]], [[[[2, 3, 5], [2, 6, 8]], [[2, 6, 7], [5, 6, 7]], [[2, 6, 9]]], [[2, 4, 7], [2, 7, 8], [5, 7, 9]], [[[2, 5, 6], [5, 8, 9]], [[2, 5, 7]], [[2, 5, 8]]]], [[[[3, 4, 5], [6, 7, 8]], [[3, 4, 7]], [[3, 7, 9], [6, 7, 9]]], [[3, 5, 7], [3, 5, 8], [3, 8, 9]], [[3, 6, 8]]]]


In [38]:
class HNode:
    """
    Class which represents node in a hash tree.
    """

    def __init__(self):
        self.children = {}
        self.isLeaf = True
        self.bucket = {}


class HTree:
    """
    Wrapper class for HTree instance
    """

    def __init__(self, max_leaf_cnt, max_child_cnt):
        self.root = HNode()
        self.max_leaf_cnt = max_leaf_cnt
        self.max_child_cnt = max_child_cnt
        self.frequent_itemsets = []

    def recur_insert(self, node, itemset, index, cnt):
        # TO-DO
        """
        Recursively adds nodes inside the tree and if required splits leaf node and
        redistributes itemsets among child converting itself into intermediate node.
        :param node:
        :param itemset:
        :param index:
        :return:
        """
        if index == len(itemset):
            # last bucket so just insert
            if itemset in node.bucket:
                node.bucket[itemset] += cnt
            else:
                node.bucket[itemset] = cnt
            return

        if node.isLeaf:

            if itemset in node.bucket:
                node.bucket[itemset] += cnt
            else:
                node.bucket[itemset] = cnt
            if len(node.bucket) == self.max_leaf_cnt:
                # bucket has reached its maximum capacity and its intermediate node so
                # split and redistribute entries.
                for old_itemset, old_cnt in node.bucket.iteritems():

                    hash_key = self.hash(old_itemset[index])
                    if hash_key not in node.children:
                        node.children[hash_key] = HNode()
                    self.recur_insert(node.children[hash_key], old_itemset, index + 1, old_cnt)
                # there is no point in having this node's bucket
                # so just delete it
                del node.bucket
                node.isLeaf = False
        else:
            hash_key = self.hash(itemset[index])
            if hash_key not in node.children:
                node.children[hash_key] = HNode()
            self.recur_insert(node.children[hash_key], itemset, index + 1, cnt)

    def insert(self, itemset):
        # as list can't be hashed we need to convert this into tuple
        # which can be easily hashed in leaf node buckets
        itemset = tuple(itemset)
        self.recur_insert(self.root, itemset, 0, 0)

    def add_support(self, itemset):
        runner = self.root
        itemset = tuple(itemset)
        index = 0
        while True:
            if runner.isLeaf:
                if itemset in runner.bucket:
                    runner.bucket[itemset] += 1
                break
            hash_key = self.hash(itemset[index])
            if hash_key in runner.children:
                runner = runner.children[hash_key]
            else:
                break
            index += 1

    def dfs(self, node, support_cnt):
        if node.isLeaf:
            for key, value in node.bucket.iteritems():
                if value >= support_cnt:
                    self.frequent_itemsets.append((list(key), value))
                    # print key, value, support_cnt
            return

        for child in node.children.values():
            self.dfs(child, support_cnt)

    def get_frequent_itemsets(self, support_cnt):
        """
        Returns all frequent itemsets which can be considered for next level
        :param support_cnt: Minimum cnt required for itemset to be considered as frequent
        :return:
        """
        self.frequent_itemsets = []
        self.dfs(self.root, support_cnt)
        return self.frequent_itemsets

    def hash(self, val):
        return val % self.max_child_cnt


def generate_hash_tree(candidate_itemsets, length, max_leaf_cnt=4, max_child_cnt=5):
    """
    This function generates hash tree of itemsets with each node having no more than child_max_length
    childs and each leaf node having no more than max_leaf_length.
    :param candidate_itemsets: Itemsets
    :param length: Length if each itemset
    :param max_leaf_length:
    :param child_max_length:
    :return:
    """
    htree = HTree(max_child_cnt, max_leaf_cnt)
    for itemset in candidate_itemsets:
        # add this itemset to hashtree
        htree.insert(itemset)
    return htree



In [39]:
h_tree = generate_hash_tree([{1,2,4}, {1,2,9}, {1,3,5}], 2)
print(h_tree)

In [45]:
import csv, itertools


def load_data(filename):
    """
    Loads transactions from given file
    :param filename:
    :return:
    """
    reader = csv.reader(open(filename, 'r'), delimiter=',')
    trans = [map(int, row[1:]) for row in reader]
    return trans


def find_frequent_one(data_set, support):
    """
    Find frequent one itemsets within data set
    :param data_set:
    :param support: Provided support value
    :return:
    """
    candidate_one = {}
    total = len(data_set)
    for row in data_set:
        for val in row:
            if val in candidate_one:
                candidate_one[val] += 1
            else:
                candidate_one[val] = 1

    frequent_1 = []
    for key, cnt in candidate_one.items():
        # check if given item has sufficient count.
        if cnt >= (support * total / 100):
            frequent_1.append(([key], cnt))
    return frequent_1


class HNode:
    """
    Class which represents node in a hash tree.
    """

    def __init__(self):
        self.children = {}
        self.isLeaf = True
        self.bucket = {}


class HTree:
    """
    Wrapper class for HTree instance
    """

    def __init__(self, max_leaf_cnt, max_child_cnt):
        self.root = HNode()
        self.max_leaf_cnt = max_leaf_cnt
        self.max_child_cnt = max_child_cnt
        self.frequent_itemsets = []

    def recur_insert(self, node, itemset, index, cnt):
        # TO-DO
        """
        Recursively adds nodes inside the tree and if required splits leaf node and
        redistributes itemsets among child converting itself into intermediate node.
        :param node:
        :param itemset:
        :param index:
        :return:
        """
        if index == len(itemset):
            # last bucket so just insert
            if itemset in node.bucket:
                node.bucket[itemset] += cnt
            else:
                node.bucket[itemset] = cnt
            return

        if node.isLeaf:

            if itemset in node.bucket:
                node.bucket[itemset] += cnt
            else:
                node.bucket[itemset] = cnt
            if len(node.bucket) == self.max_leaf_cnt:
                # bucket has reached its maximum capacity and its intermediate node so
                # split and redistribute entries.
                for old_itemset, old_cnt in node.bucket.iteritems():

                    hash_key = self.hash(old_itemset[index])
                    if hash_key not in node.children:
                        node.children[hash_key] = HNode()
                    self.recur_insert(node.children[hash_key], old_itemset, index + 1, old_cnt)
                # there is no point in having this node's bucket
                # so just delete it
                del node.bucket
                node.isLeaf = False
        else:
            hash_key = self.hash(itemset[index])
            if hash_key not in node.children:
                node.children[hash_key] = HNode()
            self.recur_insert(node.children[hash_key], itemset, index + 1, cnt)

    def insert(self, itemset):
        # as list can't be hashed we need to convert this into tuple
        # which can be easily hashed in leaf node buckets
        itemset = tuple(itemset)
        self.recur_insert(self.root, itemset, 0, 0)

    def add_support(self, itemset):
        runner = self.root
        itemset = tuple(itemset)
        index = 0
        while True:
            if runner.isLeaf:
                if itemset in runner.bucket:
                    runner.bucket[itemset] += 1
                break
            hash_key = self.hash(itemset[index])
            if hash_key in runner.children:
                runner = runner.children[hash_key]
            else:
                break
            index += 1

    def dfs(self, node, support_cnt):
        if node.isLeaf:
            for key, value in node.bucket.iteritems():
                if value >= support_cnt:
                    self.frequent_itemsets.append((list(key), value))
                    # print key, value, support_cnt
            return

        for child in node.children.values():
            self.dfs(child, support_cnt)

    def get_frequent_itemsets(self, support_cnt):
        """
        Returns all frequent itemsets which can be considered for next level
        :param support_cnt: Minimum cnt required for itemset to be considered as frequent
        :return:
        """
        self.frequent_itemsets = []
        self.dfs(self.root, support_cnt)
        return self.frequent_itemsets

    def hash(self, val):
        return val % self.max_child_cnt


def generate_hash_tree(candidate_itemsets, length, max_leaf_cnt=4, max_child_cnt=5):
    """
    This function generates hash tree of itemsets with each node having no more than child_max_length
    childs and each leaf node having no more than max_leaf_length.
    :param candidate_itemsets: Itemsets
    :param length: Length if each itemset
    :param max_leaf_length:
    :param child_max_length:
    :return:
    """
    htree = HTree(max_child_cnt, max_leaf_cnt)
    for itemset in candidate_itemsets:
        # add this itemset to hashtree
        htree.insert(itemset)
    return htree


def generate_k_subsets(dataset, length):
    subsets = []
    for itemset in dataset:
        subsets.extend(map(list, itertools.combinations(itemset, length)))
    return subsets


def is_prefix(list_1, list_2):
    for i in range(len(list_1) - 1):
        if list_1[i] != list_2[i]:
            return False
    return True


def apriori_generate_frequent_itemsets(dataset, support):
    """
    Generates frequent itemsets
    :param dataset:
    :param support:
    :return: List of f-itemsets with their respective count in
            form of list of tuples.
    """
    support_cnt = int(support / 100.0 * len(dataset))
    all_frequent_itemsets = find_frequent_one(dataset, support)
    prev_frequent = [x[0] for x in all_frequent_itemsets]
    length = 2
    while len(prev_frequent) > 1:
        new_candidates = []
        for i in range(len(prev_frequent)):
            j = i + 1
            while j < len(prev_frequent) and is_prefix(prev_frequent[i], prev_frequent[j]):
                # this part makes sure that all of the items remain lexicographically sorted.
                new_candidates.append(prev_frequent[i][:-1] +
                                      [prev_frequent[i][-1]] +
                                      [prev_frequent[j][-1]]
                                      )
                j += 1

        # generate hash tree and find frequent itemsets
        h_tree = generate_hash_tree(new_candidates, length)
        # for each transaction, find all possible subsets of size "length"
        k_subsets = generate_k_subsets(dataset, length)

        # support counting and finding frequent itemsets
        for subset in k_subsets:
            h_tree.add_support(subset)

        # find frequent itemsets
        new_frequent = h_tree.get_frequent_itemsets(support_cnt)
        all_frequent_itemsets.extend(new_frequent)
        prev_frequent = [tup[0] for tup in new_frequent]
        prev_frequent.sort()
        length += 1

    return all_frequent_itemsets


def generate_association_rules(f_itemsets, confidence):
    """
    This method generates association rules with confidence greater than threshold
    confidence. For finding confidence we don't need to traverse dataset again as we
    already have support of frequent itemsets.
    Remember Anti-monotone property ?
    I've done pruning in this step also, which reduced its complexity significantly:
    Say X -> Y is AR which don't have enough confidence then any other rule X' -> Y'
    where (X' subset of X) is not possible as sup(X') >= sup(X).
    :param f_itemsets: Frequent itemset with their support values
    :param confidence:
    :return: Returns association rules with associated confidence
    """

    hash_map = {}
    for itemset in f_itemsets:
        hash_map[tuple(itemset[0])] = itemset[1]

    a_rules = []
    for itemset in f_itemsets:
        length = len(itemset[0])
        if length == 1:
            continue

        union_support = hash_map[tuple(itemset[0])]
        for i in range(1, length):

            lefts = map(list, itertools.combinations(itemset[0], i))
            for left in lefts:
                conf = 100.0 * union_support / hash_map[tuple(left)]
                if conf >= confidence:
                    a_rules.append([left,list(set(itemset[0]) - set(left)), conf])
    return a_rules


def print_rules(rules):

    for item in rules:
        left = ','.join(map(str, item[0]))
        right = ','.join(map(str, item[1]))
        print (' ==> '.join([left, right]))
    print('Total Rules Generated: ', len(rules))

if __name__ == '__main__':
    transactions = load_data('/Users/zwangeb/Downloads/T10I4D100K.dat')
    # print find_frequent_one(transactions, 5)
    frequent = apriori_generate_frequent_itemsets(transactions, 10)
    # for item in frequent:
    #     if len(item[0]) > 1:
    #         print item

    print(frequent)

[]


In [36]:
# """
# # Python 2.7
# # Filename: apriori.py
# # Author: llhthinker
# # Email: hangliu56[AT]gmail[DOT]com
# # Blog: http://www.cnblogs.com/llhthinker/p/6719779.html
# # Date: 2017-04-16
# """
#
# import pandas as pd
# def load_data_set():
#     myData=pd.read_table('C:/Users/dell/Desktop/T10I4D100K.dat.txt',header=None)
#     myData.columns=['name']
#     print(type(myData['name'][0]))
#     print(myData['name'])
#     dataset=[]
#     for i in range(len(myData)):
#         tmp=[]
#         tmp=myData['name'][i].split(' ')
#         if '' in tmp:
#             tmp.remove('')
#         dataset.append(tmp)
#     print(dataset)
#     # data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'],
#     #             ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
#     #             ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]
#     return dataset
#
#
# def create_C1(data_set):
#     """
#     Create frequent candidate 1-itemset C1 by scaning data set.
#     Args:
#         data_set: A list of transactions. Each transaction contains several items.
#     Returns:
#         C1: A set which contains all frequent candidate 1-itemsets
#     """
#     C1 = set()
#     for t in data_set:
#         for item in t:
#             item_set = frozenset([item])
#             C1.add(item_set)
#     return C1
#
#
# def is_apriori(Ck_item, Lksub1):
#     """
#     Judge whether a frequent candidate k-itemset satisfy Apriori property.
#     Args:
#         Ck_item: a frequent candidate k-itemset in Ck which contains all frequent
#                  candidate k-itemsets.
#         Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
#     Returns:
#         True: satisfying Apriori property.
#         False: Not satisfying Apriori property.
#     """
#     for item in Ck_item:
#         sub_Ck = Ck_item - frozenset([item])
#         if sub_Ck not in Lksub1:
#             return False
#     return True
#
#
# def create_Ck(Lksub1, k):
#     """
#     Create Ck, a set which contains all all frequent candidate k-itemsets
#     by Lk-1's own connection operation.
#     Args:
#         Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
#         k: the item number of a frequent itemset.
#     Return:
#         Ck: a set which contains all all frequent candidate k-itemsets.
#     """
#     Ck = set()
#     len_Lksub1 = len(Lksub1)
#     list_Lksub1 = list(Lksub1)
#     for i in range(len_Lksub1):
#         for j in range(1, len_Lksub1):
#             l1 = list(list_Lksub1[i])
#             l2 = list(list_Lksub1[j])
#             l1.sort()
#             l2.sort()
#             if l1[0:k - 2] == l2[0:k - 2]:
#                 Ck_item = list_Lksub1[i] | list_Lksub1[j]
#                 # pruning
#                 if is_apriori(Ck_item, Lksub1):
#                     Ck.add(Ck_item)
#     return Ck
#
#
# def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
#     """
#     Generate Lk by executing a delete policy from Ck.
#     Args:
#         data_set: A list of transactions. Each transaction contains several items.
#         Ck: A set which contains all all frequent candidate k-itemsets.
#         min_support: The minimum support.
#         support_data: A dictionary. The key is frequent itemset and the value is support.
#     Returns:
#         Lk: A set which contains all all frequent k-itemsets.
#     """
#     Lk = set()
#     item_count = {}
#     for t in data_set:
#         for item in Ck:
#             if item.issubset(t):
#                 if item not in item_count:
#                     item_count[item] = 1
#                 else:
#                     item_count[item] += 1
#     t_num = float(len(data_set))
#     for item in item_count:
#         if (item_count[item] / t_num) >= min_support:
#             Lk.add(item)
#             support_data[item] = item_count[item] / t_num
#     return Lk
#
#
# def generate_L(data_set, k, min_support):
#     """
#     Generate all frequent itemsets.
#     Args:
#         data_set: A list of transactions. Each transaction contains several items.
#         k: Maximum number of items for all frequent itemsets.
#         min_support: The minimum support.
#     Returns:
#         L: The list of Lk.
#         support_data: A dictionary. The key is frequent itemset and the value is support.
#     """
#     support_data = {}
#     C1 = create_C1(data_set)
#     L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
#     Lksub1 = L1.copy()
#     L = []
#     L.append(Lksub1)
#     for i in range(2, k + 1):
#         Ci = create_Ck(Lksub1, i)
#         Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
#         Lksub1 = Li.copy()
#         L.append(Lksub1)
#     return L, support_data
#
#
# def generate_big_rules(L, support_data, min_conf):
#     """
#     Generate big rules from frequent itemsets.
#     Args:
#         L: The list of Lk.
#         support_data: A dictionary. The key is frequent itemset and the value is support.
#         min_conf: Minimal confidence.
#     Returns:
#         big_rule_list: A list which contains all big rules. Each big rule is represented
#                        as a 3-tuple.
#     """
#     big_rule_list = []
#     sub_set_list = []
#     for i in range(0, len(L)):
#         for freq_set in L[i]:
#             for sub_set in sub_set_list:
#                 if sub_set.issubset(freq_set):
#                     conf = support_data[freq_set] / support_data[freq_set - sub_set]
#                     big_rule = (freq_set - sub_set, sub_set, conf)
#                     if conf >= min_conf and big_rule not in big_rule_list:
#                         # print freq_set-sub_set, " => ", sub_set, "conf: ", conf
#                         big_rule_list.append(big_rule)
#             sub_set_list.append(freq_set)
#     return big_rule_list
#
# import datetime
# if __name__ == "__main__":
#     """
#     Test
#     """
#     start=datetime.datetime.now()
#     data_set = load_data_set()
#     L, support_data = generate_L(data_set, k=3, min_support=0.01)#0.2
#     big_rules_list = generate_big_rules(L, support_data, min_conf=0.1)#0.7
#     print('Lk is',L)
#     for Lk in L:
#         print("=" * 50)
#         print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
#         print("=" * 50)
#         for freq_set in Lk:
#             print(freq_set, support_data[freq_set])
#
#     print("Big Rules")
#     for item in big_rules_list:
#         print(item[0], "=>", item[1], "conf: ", item[2])
#     end=datetime.datetime.now()
#     print('time is',end-start)

import itertools
import time

# take input of file name and minimum support count
print("Enter the filename:")
filename = input()
print("Enter the minimum support count:")
min_support = int(input())

# read data from txt file
with open(filename) as f:
    content = f.readlines()

content = [x.strip() for x in content]

Transaction = []  # to store transaction
Frequent_items_value = {}  # to store all frequent item sets

# to fill values in transaction from txt file
for i in range(0, len(content)):
    Transaction.append(content[i].split())


# function to get frequent one itemset
def frequent_one_item(Transaction, min_support):
    candidate1 = {}

    for i in range(0, len(Transaction)):
        for j in range(0, len(Transaction[i])):
            if Transaction[i][j] not in candidate1:
                candidate1[Transaction[i][j]] = 1
            else:
                candidate1[Transaction[i][j]] += 1

    frequentitem1 = []  # to get frequent 1 itemsets with minimum support count
    for value in candidate1:
        if candidate1[value] >= min_support:
            frequentitem1 = frequentitem1 + [[value]]
            Frequent_items_value[tuple(value)] = candidate1[value]

    return frequentitem1


values = frequent_one_item(Transaction, min_support)
print(values)
print(Frequent_items_value)

# to remove infrequent 1 itemsets from transaction
Transaction1 = []
for i in range(0, len(Transaction)):
    list_val = []
    for j in range(0, len(Transaction[i])):
        if [Transaction[i][j]] in values:
            list_val.append(Transaction[i][j])
    Transaction1.append(list_val)


# class of Hash node
class Hash_node:
    def __init__(self):
        self.children = {}  # pointer to its children
        self.Leaf_status = True  # to know the status whether current node is leaf or not
        self.bucket = {}  # contains itemsets in bucket


# class of constructing and getting hashtree
class HashTree:
    # class constructor
    def __init__(self, max_leaf_count, max_child_count):
        self.root = Hash_node()
        self.max_leaf_count = max_leaf_count
        self.max_child_count = max_child_count
        self.frequent_itemsets = []

    # function to recursive insertion to make hashtree
    def recursively_insert(self, node, itemset, index, count):
        if index == len(itemset):
            if itemset in node.bucket:
                node.bucket[itemset] += count
            else:
                node.bucket[itemset] = count
            return

        if node.Leaf_status:  # if node is leaf
            if itemset in node.bucket:
                node.bucket[itemset] += count
            else:
                node.bucket[itemset] = count
            if len(node.bucket) == self.max_leaf_count:  # if bucket capacity increases
                for old_itemset, old_count in node.bucket.items():

                    hash_key = self.hash_function(old_itemset[index])  # do hashing on next index
                    if hash_key not in node.children:
                        node.children[hash_key] = Hash_node()
                    self.recursively_insert(node.children[hash_key], old_itemset, index + 1, old_count)
                # since no more requirement of this bucket
                del node.bucket
                node.Leaf_status = False
        else:  # if node is not leaf
            hash_key = self.hash_function(itemset[index])
            if hash_key not in node.children:
                node.children[hash_key] = Hash_node()
            self.recursively_insert(node.children[hash_key], itemset, index + 1, count)

    def insert(self, itemset):
        itemset = tuple(itemset)
        self.recursively_insert(self.root, itemset, 0, 0)

    # to add support to candidate itemsets. Transverse the Tree and find the bucket in which this itemset is present.
    def add_support(self, itemset):
        Transverse_HNode = self.root
        itemset = tuple(itemset)
        index = 0
        while True:
            if Transverse_HNode.Leaf_status:
                if itemset in Transverse_HNode.bucket:  # found the itemset in this bucket
                    Transverse_HNode.bucket[itemset] += 1  # increment the count of this itemset.
                break
            hash_key = self.hash_function(itemset[index])
            if hash_key in Transverse_HNode.children:
                Transverse_HNode = Transverse_HNode.children[hash_key]
            else:
                break
            index += 1

    # to transverse the hashtree to get frequent itemsets with minimum support count
    def get_frequent_itemsets(self, node, support_count, frequent_itemsets):
        if node.Leaf_status:
            for key, value in node.bucket.items():
                if value >= support_count:  # if it satisfies the condition
                    frequent_itemsets.append(list(key))  # then add it to frequent itemsets.
                    Frequent_items_value[key] = value
            return

        for child in node.children.values():
            self.get_frequent_itemsets(child, support_count, frequent_itemsets)

    # hash function for making HashTree
    def hash_function(self, val):
        return int(val) % self.max_child_count


# To generate hash tree from candidate itemsets
def generate_hash_tree(candidate_itemsets, max_leaf_count, max_child_count):
    htree = HashTree(max_child_count, max_leaf_count)  # create instance of HashTree
    for itemset in candidate_itemsets:
        htree.insert(itemset)  # to insert itemset into Hashtree
    return htree


# to generate subsets of itemsets of size k
def generate_k_subsets(dataset, length):
    subsets = []
    for itemset in dataset:
        subsets.extend(map(list, itertools.combinations(itemset, length)))
    return subsets


def subset_generation(ck_data, l):
    return map(list, set(itertools.combinations(ck_data, l)))


# apriori generate function to generate ck
def apriori_generate(dataset, k):
    ck = []
    # join step
    lenlk = len(dataset)
    for i in range(lenlk):
        for j in range(i + 1, lenlk):
            L1 = list(dataset[i])[:k - 2]
            L2 = list(dataset[j])[:k - 2]
            if L1 == L2:
                ck.append(sorted(list(set(dataset[i]) | set(dataset[j]))))

    # prune step
    final_ck = []
    for candidate in ck:
        all_subsets = list(subset_generation(set(candidate), k - 1))
        found = True
        for i in range(len(all_subsets)):
            value = list(sorted(all_subsets[i]))
            if value not in dataset:
                found = False
        if found == True:
            final_ck.append(candidate)

    return ck, final_ck


def generateL(ck, min_support):
    support_ck = {}
    for val in Transaction1:
        for val1 in ck:
            value = set(val)
            value1 = set(val1)

            if value1.issubset(value):
                if tuple(val1) not in support_ck:
                    support_ck[tuple(val1)] = 1
                else:
                    support_ck[tuple(val1)] += 1
    frequent_item = []
    for item_set in support_ck:
        if support_ck[item_set] >= min_support:
            frequent_item.append(sorted(list(item_set)))
            Frequent_items_value[item_set] = support_ck[item_set]

    return frequent_item


# main apriori algorithm function
def apriori(L1, min_support):
    k = 2;
    L = []
    L.append(0)
    L.append(L1)
    print("enter max_leaf_count")  # maximum number of items in bucket i.e. bucket capacity of each node
    max_leaf_count = int(input())
    print("enter max_child_count")  # maximum number of child you want for a node
    max_child_count = int(input())

    start = time.time()
    while (len(L[k - 1]) > 0):
        ck, final_ck = apriori_generate(L[k - 1], k)  # to generate candidate itemsets
        print("C%d" % (k))
        print(final_ck)
        h_tree = generate_hash_tree(ck, max_leaf_count, max_child_count)  # to generate hashtree
        if (k > 2):
            while (len(L[k - 1]) > 0):
                l = generateL(final_ck, min_support)
                L.append(l)
                print("Frequent %d item" % (k))
                print(l)
                k = k + 1
                ck, final_ck = apriori_generate(L[k - 1], k)
                print("C%d" % (k))
                print(final_ck)
            break
        k_subsets = generate_k_subsets(Transaction1, k)  # to generate subsets of each transaction
        for subset in k_subsets:
            h_tree.add_support(subset)  # to add support count to itemsets in hashtree
        lk = []
        h_tree.get_frequent_itemsets(h_tree.root, min_support, lk)  # to get frequent itemsets
        print("Frequent %d item" % (k))
        print(lk)
        L.append(lk)
        k = k + 1
    end = time.time()
    return L, (end - start)


L_value, time_taken = apriori(values, min_support)
print("Time Taken is:")
print(time_taken)
# print("final L_value")
# print(L_value)
print("All frequent itemsets with their support count:")
print(Frequent_items_value)


Enter the filename:
/Users/zwangeb/Downloads/T10I4D100K.dat
Enter the minimum support count:
1000
[['52'], ['240'], ['274'], ['368'], ['448'], ['538'], ['561'], ['630'], ['687'], ['775'], ['825'], ['120'], ['205'], ['401'], ['581'], ['704'], ['814'], ['674'], ['733'], ['854'], ['422'], ['449'], ['857'], ['895'], ['937'], ['229'], ['283'], ['294'], ['381'], ['708'], ['738'], ['766'], ['853'], ['883'], ['966'], ['104'], ['143'], ['569'], ['620'], ['185'], ['214'], ['350'], ['529'], ['658'], ['682'], ['782'], ['809'], ['947'], ['970'], ['192'], ['208'], ['279'], ['280'], ['496'], ['530'], ['597'], ['618'], ['675'], ['720'], ['914'], ['932'], ['217'], ['276'], ['653'], ['706'], ['878'], ['175'], ['177'], ['424'], ['490'], ['571'], ['623'], ['795'], ['910'], ['960'], ['130'], ['461'], ['862'], ['78'], ['900'], ['921'], ['147'], ['411'], ['572'], ['579'], ['778'], ['803'], ['266'], ['290'], ['458'], ['523'], ['614'], ['888'], ['944'], ['70'], ['204'], ['227'], ['334'], ['480'], ['513'], ['87

enter max_leaf_count
1000
enter max_child_count
10
C2
[['240', '52'], ['274', '52'], ['368', '52'], ['448', '52'], ['52', '538'], ['52', '561'], ['52', '630'], ['52', '687'], ['52', '775'], ['52', '825'], ['120', '52'], ['205', '52'], ['401', '52'], ['52', '581'], ['52', '704'], ['52', '814'], ['52', '674'], ['52', '733'], ['52', '854'], ['422', '52'], ['449', '52'], ['52', '857'], ['52', '895'], ['52', '937'], ['229', '52'], ['283', '52'], ['294', '52'], ['381', '52'], ['52', '708'], ['52', '738'], ['52', '766'], ['52', '853'], ['52', '883'], ['52', '966'], ['104', '52'], ['143', '52'], ['52', '569'], ['52', '620'], ['185', '52'], ['214', '52'], ['350', '52'], ['52', '529'], ['52', '658'], ['52', '682'], ['52', '782'], ['52', '809'], ['52', '947'], ['52', '970'], ['192', '52'], ['208', '52'], ['279', '52'], ['280', '52'], ['496', '52'], ['52', '530'], ['52', '597'], ['52', '618'], ['52', '675'], ['52', '720'], ['52', '914'], ['52', '932'], ['217', '52'], ['276', '52'], ['52', '653'], 

Frequent 2 item
[['368', '682'], ['368', '829'], ['217', '346'], ['227', '390'], ['39', '825'], ['39', '704'], ['390', '722'], ['704', '825'], ['789', '829']]
C3
[['39', '704', '825']]
Frequent 3 item
[['39', '704', '825']]
C4
[]
Frequent 4 item
[]
C5
[]
Time Taken is:
12.197052001953125
All frequent itemsets with their support count:
{('5', '2'): 1982, ('2', '4', '0'): 1399, ('2', '7', '4'): 2628, ('3', '6', '8'): 7828, ('4', '4', '8'): 1370, ('5', '3', '8'): 3982, ('5', '6', '1'): 2782, ('6', '3', '0'): 1523, ('6', '8', '7'): 1762, ('7', '7', '5'): 3771, ('8', '2', '5'): 3085, ('1', '2', '0'): 4973, ('2', '0', '5'): 3605, ('4', '0', '1'): 3667, ('5', '8', '1'): 2943, ('7', '0', '4'): 1794, ('8', '1', '4'): 1672, ('6', '7', '4'): 2527, ('7', '3', '3'): 1141, ('8', '5', '4'): 2847, ('4', '2', '2'): 1255, ('4', '4', '9'): 1890, ('8', '5', '7'): 1588, ('8', '9', '5'): 3385, ('9', '3', '7'): 4681, ('2', '2', '9'): 2281, ('2', '8', '3'): 4082, ('2', '9', '4'): 1445, ('3', '8', '1'): 2959, 